In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import json
from geopy.geocoders import Nominatim
from crime_clustering import CrimeCluster
from geopy.distance import great_circle
import pickle5 as pickle
from sklearn.linear_model import LinearRegression
import base64
from zillowscraper import address_to_price_and_image
import requests

## Mapbox Token
The mapbox token is requried to generate the charts that contain geographic data.

In [2]:
mapbox_token = open('tokens/mapbox-token.txt').read()
px.set_mapbox_access_token(mapbox_token)

## Crime Data
The cleaned crime data is loaded into a pandas dataframe and filtered for all data before the year 2021.

In [3]:
df = pd.read_csv('data/crime-clean.csv')
df = df[(df['Year'] < 2021)]

In [4]:
df.sample(15)

,Crime Type,Latitude,Longitude,Neighborhood,Zip Code,Crime Score,Neigh Score,CSperCapita,Year,Month
2177800,THEFT,41.950319,-87.776067,Portage Park,60634,0.3261,1313.7834,0.022225,2018,4
2209182,THEFT,41.951935,-87.790854,Dunning,60634,0.0000,2578.6960,0.043623,2013,11
426286,DECEPTIVE PRACTICE,41.873907,-87.725430,Garfield Park,60624,0.1703,16713.8736,0.670728,2018,8
411192,NARCOTICS,41.878027,-87.721906,Garfield Park,60624,0.2138,16713.8736,0.670728,2016,7
2775105,THEFT,41.894842,-87.630554,River North,60654,0.0000,4591.3610,0.254482,2014,9
2628310,THEFT,41.999663,-87.709364,West Ridge,60645,0.0000,3287.3964,0.094880,2011,5
907801,THEFT,41.793067,-87.671840,Englewood,60636,0.2637,13534.0511,0.562840,2017,8
1403666,CRIMINAL DAMAGE,41.882025,-87.692766,Garfield Park,60612,0.0000,4741.9232,0.182206,2012,12
2584812,BATTERY,41.963753,-87.721856,Albany Park,60625,0.0000,3625.6688,0.057156,2011,7
2935514,DECEPTIVE PRACTICE,41.977056,-87.659885,Edgewater,60640,0.0000,1147.8162,0.019053,2013,8


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3149595 entries, 0 to 3187329
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Crime Type    object 
 1   Latitude      float64
 2   Longitude     float64
 3   Neighborhood  object 
 4   Zip Code      int64  
 5   Crime Score   float64
 6   Neigh Score   float64
 7   CSperCapita   float64
 8   Year          int64  
 9   Month         int64  
dtypes: float64(5), int64(3), object(2)
memory usage: 264.3+ MB


In [6]:
all_crimes = 'ALL'

The crime_score function is used to aggregate data by the crime type and location. The location is either 'Neighborhood' or 'Zip Code'. The average Crime Score and CSperCapita is calculated for each row in the aggregated data set. The data is stored in a dataframe that is used in the Choropleth map. The pre-aggregated data allows the dashboard to resond quickly to user actions.

In [7]:
def crime_score(location_column):
    
    group = [location_column]
    group_type = ['Crime Type', location_column]
     
    df_location = df.groupby(group, as_index=False).agg({'Crime Score': 'mean', 'CSperCapita': 'mean'})
    df_location['Crime Type'] = all_crimes
    
    df_location_type = df.groupby(group_type, as_index=False).agg({'Crime Score': 'mean', 'CSperCapita': 'mean'})
    
    df_location_all = pd.concat([df_location, df_location_type], axis=0)
    
    return df_location_all

Aggregated by neighborhood.

In [8]:
df_neigh = crime_score('Neighborhood')
df_neigh.head()

,Neighborhood,Crime Score,CSperCapita,Crime Type
0,Albany Park,0.150479,0.049411,ALL
1,Andersonville,0.152534,0.009535,ALL
2,Archer Heights,0.135140,0.024384,ALL
3,Armour Square,0.141911,0.011887,ALL
4,Ashburn,0.147052,0.120671,ALL


Aggregated by zip code.

In [9]:
df_zip = crime_score('Zip Code')
df_zip.head()

,Zip Code,Crime Score,CSperCapita,Crime Type
0,60601,0.175033,0.259871,ALL
1,60602,0.164967,2.522610,ALL
2,60603,0.173598,1.766483,ALL
3,60604,0.153269,1.762450,ALL
4,60605,0.160981,0.059538,ALL


The agg_crime function aggregates the crime data by the year, crime Type, and location. The count of each type of crime is calculated for each row in the aggregated data set. This data is also pre-aggregated to allow the dashboard to respond quickly.

In [10]:
def agg_crime(location_column=None):
    
    if location_column is None:
        group = ['Year']
        group_type = ['Crime Type', 'Year']
    else:
        group = ['Year', location_column]
        group_type = ['Crime Type' , 'Year', location_column]
    
    df_location = df.groupby(group, as_index=False)\
                        .agg({'Latitude': 'count'}).rename(columns={'Latitude': 'Count'})
    df_location['Crime Type'] = all_crimes
    df_location_type = df.groupby(group_type, as_index=False)\
                            .agg({'Latitude': 'count'}).rename(columns={'Latitude': 'Count'})
    df_location_all = pd.concat([df_location, df_location_type], axis=0)
    return df_location_all

Aggregated for the entired city of Chicago.

In [11]:
df_crime_chicago = agg_crime()
df_crime_chicago.head()

,Year,Count,Crime Type
0,2010,369008,ALL
1,2011,350350,ALL
2,2012,334599,ALL
3,2013,305572,ALL
4,2014,273045,ALL


Aggregated by neighborhood.

In [12]:
df_crime_neigh = agg_crime('Neighborhood')
df_crime_neigh.head()

,Year,Neighborhood,Count,Crime Type
0,2010,Albany Park,3491,ALL
1,2010,Andersonville,436,ALL
2,2010,Archer Heights,1358,ALL
3,2010,Armour Square,783,ALL
4,2010,Ashburn,3447,ALL


Aggregated by zip code.

In [13]:
df_crime_zip = agg_crime('Zip Code')
df_crime_zip.head()

,Year,Zip Code,Count,Crime Type
0,2010,60601,1567,ALL
1,2010,60602,1818,ALL
2,2010,60603,884,ALL
3,2010,60604,1015,ALL
4,2010,60605,2727,ALL


## Geographic Data
The geographic data for both the neighborhoods and zip codes for Chicago are stored in geojson files. The geographic data is used to outline the locations on the Chloropleth map and determine what location each crime in Chicago was commited.

In [14]:
with open('data/geo/Neighborhoods.geojson') as Neigh:
    geodict = {'json_neigh': json.load(Neigh)}

In [15]:
with open('data/geo/ZIP.geojson') as ZIP22:
    geodict['json_zip'] = json.load(ZIP22)

## Crime Cluster
The CrimeCluster object is initialized with the crime data and geographic locations data. The CrimeCluster object will used that data to find crime clusters in specified locations.

In [16]:
cCluster = CrimeCluster(df, geodict['json_neigh'], geodict['json_zip'])

## Crime Types
The uniqe crime types are determined from the crime data to use in a drop down in the dashboard. This will allow the user to filer the data set by the different types of crime.

In [17]:
crime_types = [all_crimes] + list(df['Crime Type'].unique())

## Crime Clusters
The crime cluster data for the entire city of Chicago is pre-determined. This allows the dashboard to respond quickly when data for all of Chicago is being analyzed. The cluster data is stored in a dataframe.

In [18]:
with open('data/clusters/chicago/crime_types_clusters.pickle', 'rb') as handle:
    chicago_cluster_data = pickle.load(handle)

In [19]:
df_chicago_clusters = pd.DataFrame.from_dict(chicago_cluster_data)

The css for the dashboard is loaded.

In [20]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# css documentation at https://codepen.io/chriddyp/pen/bWLwgP

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

The src_image take an image location and return a source format that can be used in the dashboard.

In [21]:
def src_image(src, URL=False):
    if not URL:
        encoded = base64.b64encode(open(src, 'rb').read())
    else:
        encoded = base64.b64encode(requests.get(src).content)
    return 'data:image/png;base64,{}'.format(encoded.decode())

## Dashboard
The dashboard html is creatd using Dash. The elements that the callback functions interact with are marked with unique ids. The form_address object contains the html elements for the user to input an address and the radius in miles around the address to peform analys on the crimes in the area.

In [22]:
form_address = html.Div(children=[
    html.Div(
        className='row',
        children = [
            html.Div(
                className='six columns',
                children = [
                    html.Label("Address", htmlFor="input-address", id="label-address"),
                    dcc.Input(id="input-address", type="search", placeholder="Address", className='u-full-width')
                ]
            ),
            html.Div(
                className='six columns',
                children = [
                    html.Label("Miles", htmlFor='input-miles', id='label-miles'),
                    dcc.Input(id='input-miles', type='number', value=1, className='u-full-width')
                ]
            )
        ]
    ),
    html.Div(
        className='row',
        children=[
            html.Button('Submit', id='btn-submit-address', className='button-primary', style={'margin-top': '10px'})
        ]
    )
])

The form_chart_options object contains the html elements that allow the user to change what data is displayed on the charts in the dashboard.

In [23]:
form_chart_options = html.Div(children=[
    html.Div(
        className='row',
        children=[
            html.Div(
                className='six columns',
                children=[
                    html.Label("Crime Type", htmlFor='crime-types', id='label-crime-types'),
                    dcc.Dropdown(id='crime-types', className='u-full-width',
                        options = [
                            {'label': i, 'value': i} for i in crime_types
                        ],
                        value = 'ALL')
                ]
            )
        ]
    ),
    html.Div(
        className='row',
        children = [
            dcc.RadioItems(id='location-select',
                options = [{'label': i, 'value': i} for i in ['Neighborhood', 'Zip Code']],
                value = 'Neighborhood',
                labelStyle = {'display': 'inline-block'})
        ]
    )
])

The html_zillow object contains the html elements that will hold the Zillow data.

In [24]:
html_zillow = html.Div(
    children=[
        html.Div(
            id='house-image',
            children=[],
            style={'height': 211}
        ),
        html.Div(id='price', children=[])
    ]
)

The app.layout will contain all of the html for the dashboard. The html objects created earlier are used along with various graph objects from the Dash Core Components.

In [25]:
app.layout = html.Div([
    html.Div(
        className="row",
        children = [
            html.H1("Chicago Crime & Real Estate", style={'text-align': 'center'})
        ]
    ),
    html.Div(
        className="row",
        children = [
            html.Div(
                className="six columns",
                children = [
                    form_address,
                    html.Br(),
                    form_chart_options,
                    dcc.Graph(id='chicago-map')
                ]
            ),
            html.Div(
                className="six columns",
                children = [
                    html_zillow,
                    dcc.Graph(id='crime-chart'),
                    dcc.Graph(id='crime-cluster-map')
                ]
            )
        ]
    ),
    html.Div(
        id='meta-data',
        className='row',
        children=[
            html.Div(id='crime-clusters', style={'display': 'none'}),
            html.Div(id='callback-data', style={'display': 'none'})
        ]
    )
])

## Callback Functions
Severall callback functions are implemented that allow the user to interact with the dashboard. The update_chicago_map callback will change the data displayed in the Choropleth map depending on the crime type selected and the location type selected.

In [26]:
@app.callback(
    Output('chicago-map', 'figure'),
    [Input('location-select', 'value'),
    Input('crime-types', 'value')]
)
def update_chicago_map(value, crime_type):

    if value == 'Neighborhood':
        data_frame = df_neigh[df_neigh['Crime Type'] == crime_type]
        locations = 'Neighborhood'
        geojson = geodict['json_neigh']
        featureidkey = 'properties.pri_neigh'
    else:
        data_frame = df_zip[df_zip['Crime Type'] == crime_type]
        locations = 'Zip Code'
        geojson = geodict['json_zip']
        featureidkey = 'properties.zip'

    fig = px.choropleth_mapbox(
        data_frame = data_frame,
        locations = locations,
        geojson = geojson,
        featureidkey = featureidkey,
        mapbox_style = 'carto-positron',
        color = 'Crime Score',
        center = {'lat': 41.84, 'lon': -87.6298},
        zoom = 9,
        opacity = 0.5,
        labels = {'Weighted Score': 'CSperCapita'},
        height=800)

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    return fig

The ellipses function creates the ellipses objects that will represent clusters of crime.

In [27]:
def ellipses(means, covariances, n_per_cluster=None):
    ellipse_layers = []
    if n_per_cluster is None:
        n_per_cluster = np.ones(covariances.shape[0])
    else:
        max_crimes = np.max(n_per_cluster)
        n_per_cluster /= max_crimes
        n_per_cluster[n_per_cluster < 0.25] = 0.25
    t = np.linspace(0, 2*np.pi, 20)
    for i in range(covariances.shape[0]):
        covariance_matrix = covariances[i]
        center_x, center_y = means.lon[i], means.lat[i]
        opacity_scale = n_per_cluster[i]
        v, w = np.linalg.eigh(covariance_matrix)
        u = w[0] / np.linalg.norm(w[0])
        shift_angle = np.arctan2(u[1], u[0])
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        a = v[0]
        b = v[1]
        for i in range(40):
            scale_factor = i / 40
            x = center_x + a * b / np.sqrt((b * np.cos(t))**2 + (a * np.sin(t))**2) * np.sin(t + shift_angle) * scale_factor
            y = center_y + a * b / np.sqrt((b * np.cos(t))**2 + (a * np.sin(t))**2) * np.cos(t + shift_angle) * scale_factor
            green = 255 * scale_factor
            coords=[]
            for lon, lat in zip(list(x), list(y)):
                coords.append([lon, lat]) 
            layer = dict(sourcetype = 'geojson',
                         source={ "type": "Feature",
                                 "geometry": {"type": "LineString",
                                              "coordinates": coords
                                              }
                                },
                         color = 'rgb(255,' + str(int(green)) + ', 0)',
                         type = 'line',
                         below = '',
                         opacity = (1 - scale_factor) * opacity_scale,
                         line = dict(width=2 * opacity_scale),
                        )
            ellipse_layers.append(layer)
    return ellipse_layers

The update_data function loads a dictionary with certain data from the dashboard. The dictionary data is used to determine which elements of the dashboard the user has interacted with.

In [28]:
def update_data(location, location_type, n_clicks, crime_type):
    data = {
        'n_clicks': n_clicks,
        'Location Type': location_type,
        'Location': location,
        'Crime Type': crime_type
    }
    
    return data

The line_chart function generates a line chart figure for the dashboard.

In [29]:
def line_chart(df, title):
    
    df_copy = df.copy()
    reg = LinearRegression().fit(df_copy['Year'].values.reshape((-1,1)), df_copy['Count'])
    trend = reg.predict(df_copy['Year'].values.reshape((-1,1)))
    
    dict_trend = {
        'Year': df_copy['Year'].to_list(),
        'Count': list(trend)
    }
    
    df_trend = pd.DataFrame.from_dict(dict_trend)
    
    df_copy['Data'] = 'Number of Crimes'
    df_trend['Data'] = 'Trend'
    
    df_copy = pd.concat([df_copy, df_trend], axis=0)
    df_copy.rename(columns={'Count': 'Number of Crimes'}, inplace=True)
    
    fig_chart = px.line(
        data_frame = df_copy,
        x = 'Year',
        y = 'Number of Crimes',
        color = 'Data',
        title = title
    )
    
    max_count = df_copy['Number of Crimes'].max()
    
    fig_chart.update_layout(yaxis_range=[0, max_count * 1.1])
    
    return fig_chart

The charts_address function generates the clusters chart and line chart for the data in the radius around the address specified.

In [30]:
def charts_address(address, miles, crime_type):
    gmm_data = cCluster.GMM_Address(address, miles, crime_type)
    num_centers = gmm_data[0]
    df_centers = pd.DataFrame(gmm_data[1], columns=['lat', 'lon'])
    covariances = gmm_data[2]
    n_per_cluster = gmm_data[4]
    location = gmm_data[5]
    lat = location.latitude
    lon = location.longitude
    
#     df_centers['size'] = 1
    
    df_address = pd.DataFrame.from_dict({
        'lat': [location.latitude],
        'lon': [location.longitude]
    })
    
#     df_address['size'] = 100

    fig_map = px.scatter_mapbox(
            df_address,
            lat='lat',
            lon='lon',
            zoom=12,
            mapbox_style='carto-positron',
            size=[3],
        )
    
    ellipse_layers = ellipses(df_centers, covariances, n_per_cluster)
#     df_centers = pd.concat([df_centers, df_address], axis=0)
#     df_centers.reset_index(drop=True, inplace=True)



    fig_map.layout.update(mapbox_layers=ellipse_layers)
    
    df_crime_filtered = gmm_data[3]
    
    df_chart = df_crime_filtered.groupby(['Year'], as_index=False)\
        .agg({'Latitude': 'count'}).rename(columns={'Latitude': 'Count'})

    fig_chart = line_chart(df_chart, 'Address: {}'.format(address))

    return fig_chart, fig_map, df_centers.to_json()

The charts_neigh_zip function generates the clusters chart and line chart for the data in the neighborhood or zip code selected.

In [31]:
def charts_neigh_zip(location, locationSelectValue, crime_type):
    if locationSelectValue == 'Neighborhood':
        df_crime_chart = df_crime_neigh[df_crime_neigh['Crime Type'] == crime_type]
        column = 'Neighborhood'
    else:
        df_crime_chart = df_crime_zip[df_crime_zip['Crime Type'] == crime_type]
        column = 'Zip Code'

    # location = clickData['points'][0]['location']
    df_crime_chart = df_crime_chart[(df_crime_chart[column] == location)]

    gmm_data = cCluster.GMM(location, crime_type)
    num_centers = gmm_data[0]
    df_centers = pd.DataFrame(gmm_data[1], columns=['lat', 'lon'])
    covariances = gmm_data[2]

    fig_map = px.scatter_mapbox(
        df_centers,
        lat='lat',
        lon='lon',
        zoom=12,
        mapbox_style='carto-positron'
    )

    ellipse_layers = ellipses(df_centers, covariances)
    fig_map.layout.update(mapbox_layers=ellipse_layers)

    fig_chart = line_chart(df_crime_chart, '{0}: {1}'.format(column, location))
    
    return fig_chart, fig_map, df_centers.to_json()

The charts_chicago function generates the clusters chart and line chart for all of Chicago.

In [32]:
def charts_chicago(crime_type):
    # gmm_data = cCluster.GMM_Chicago(crime_type)
    
    gmm_data = df_chicago_clusters[df_chicago_clusters['Crime Type'] == crime_type].to_dict(orient='records')[0]
    num_centers = gmm_data['num centers']
    df_centers = pd.DataFrame(gmm_data['centers'], columns=['lat', 'lon'])
    covariances = gmm_data['covariances']


    fig_map = px.scatter_mapbox(
        df_centers,
        lat='lat',
        lon='lon',
        zoom=9,
        mapbox_style='carto-positron'
    )

    ellipse_layers = ellipses(df_centers, covariances)
    fig_map.layout.update(mapbox_layers=ellipse_layers)
    
    df_crime_chicago_filtered = df_crime_chicago[df_crime_chicago['Crime Type'] == crime_type]

    fig_chart = line_chart(df_crime_chicago_filtered, 'Chicago')

    return fig_chart, fig_map, df_centers.to_json()

The update_crime_chart callback function handles when the user interacts with address inputs, the crime type selections, the location type selection, and selecting locations on the Choropleth map. It will update the charts according to what the user inputs or selects.

In [33]:
@app.callback(
    [Output('crime-chart', 'figure'),
    Output('crime-cluster-map', 'figure'),
    Output('crime-clusters', 'children'),
    Output('callback-data', 'children'),
    Output('house-image', 'children'),
    Output('price', 'children')],
    [Input('chicago-map', 'clickData'),
    Input('location-select', 'value'),
    Input('btn-submit-address', 'n_clicks'),
    Input('crime-types', 'value')],
    [State('input-address', 'value'),
    State('input-miles', 'value'),
    State('callback-data', 'children'),
    State('house-image', 'children'),
    State('price', 'children')]
)
def update_crime_chart(clickData, locationSelectValue, n_clicks, crime_type, address, miles, data, img, price):
    
    if data is None:
        data = update_data(clickData, locationSelectValue, n_clicks, crime_type)
        fresh_load = True
    else:
        data = json.loads(data)
        fresh_load = False
        
    if fresh_load:
        # Fresh load of the dashboard.
        charts = charts_chicago(crime_type)
        data = update_data('Chicago', locationSelectValue, n_clicks, crime_type)
        
    if n_clicks != data['n_clicks']:
        # Submit button was clicked.
        charts = charts_address(address, miles, crime_type)
        house_data = address_to_price_and_image(address)
        if house_data == 'INVALID ADDRESS':
            #handle invalid address here
            img = html.Div(style={'height': 200})
            price = 'INVALID ADDRESS'
        elif house_data == 'HOUSE NOT FOUND':
            #handle house not on Zillow
            img = html.Div(style={'height': 200})
            price = 'NO ZILLOW LISTING'
        else:
            price = 'Zestimate: {}'.format(house_data[0])
            image_src = house_data[1]
            img = html.Img(src=src_image(image_src, URL=True), style={'height': 200})
            data = update_data(address, 'Address', n_clicks, crime_type)
        
    
    elif crime_type != data['Crime Type']:
        # The crime type was changed.
        if data['Location Type'] == 'Address':
            charts = charts_address(address, miles, crime_type)
            data = update_data(address, 'Address', n_clicks, crime_type, src)
        elif data['Location'] == 'Chicago':
            charts = charts_chicago(crime_type)
            data = update_data('Chicago', locationSelectValue, n_clicks, crime_type)
        else:
            location = data['Location']
            location_type = data['Location Type']
            charts = charts_neigh_zip(location, location_type, crime_type)
            data = update_data(location, location_type, n_clicks, crime_type, src)
    
    elif locationSelectValue != data['Location Type'] and crime_type == data['Crime Type']:
        charts = charts_chicago(crime_type)
        data = update_data('Chicago', locationSelectValue, n_clicks, crime_type)
            
    elif clickData is not None:
        # Either a neighborhood or zip code was clicked.
        location = clickData['points'][0]['location']
        location_type = locationSelectValue
        charts = charts_neigh_zip(location, location_type, crime_type)
        data = update_data(location, location_type, n_clicks, crime_type)
    
    else:
        charts = charts_chicago(crime_type)
        data = update_data('Chicago', locationSelectValue, n_clicks, crime_type)
        
    values = list(charts)
    values.append(json.dumps(data))
    values.append(img)
    values.append(price)
        
    return tuple(values)

Starts up the server.

In [34]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Apr/2021 15:25:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 15:25:42] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1619381845.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 15:25:42] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1619381845.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 15:25:42] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1619381845.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 15:25:42] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1619381845.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 15:25:42] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_3m1619381845.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 15:25:42] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_16_0m1619381845.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 15